In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModel, AutoTokenizer
from experimental.t5_dataset import T5Dataset
import json
import os
import os.path as op
from processing.vocab import VocabSummaryDataset

st_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-bert-base-dot-v5") #model_args={"embedding_size":512})
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")


In [ ]:

DATA_DIR = "DialogSum_Data"

SAMPLE_DATA = op.join(DATA_DIR, 'dialogsum.sample.jsonl')
TEST_DATA = op.join(DATA_DIR, 'dialogsum.test.jsonl')
TRAIN_DATA = op.join(DATA_DIR, 'dialogsum.train.jsonl')
DEV_DATA = op.join(DATA_DIR, 'dialogsum.dev.jsonl')

def load_jsonl(filepath):
    
    output_list = []
    
    with open(filepath) as sd_file:
        lines = sd_file.readlines()
    
    for line in lines:
        output_list.append(json.loads(line))

    return output_list


if __name__ == "__main__":
    dev_data_list = load_jsonl(DEV_DATA)
    dev_dataset = T5Dataset(dev_data_list, 
                  summary_tokenizer=st_tokenizer,
                  debug=True)   
    for i in dev_dataset:
        pass
